# Tengu Testing

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde, pearsonr
from functools import partial

from astropy.table import Table
from astropy.io import fits

import galsim
import ngmix

import os
import time

In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
#from keras.backend.tensorflow_backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

#keras only after gpu ID and memory usage is set
import tensorflow.keras.models as km
import tensorflow.keras.layers as kl
import tensorflow.keras.regularizers as kr
import tensorflow.keras.backend as K
import tensorflow.keras.optimizers as ko

#from keras.utils.training_utils import multi_gpu_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Input, Dense, Activation
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D

print("Tensorflow version: {}".format(tf.version.VERSION))

Tensorflow version: 2.4.1


In [3]:
import tensorflow_datasets as tfds

In [4]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [5]:
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6
)

Epoch 1/6
469/469 [==============================] - 4s 3ms/step - loss: 14.1041 - sparse_categorical_accuracy: 0.7877
Epoch 2/6
469/469 [==============================] - 2s 3ms/step - loss: 0.7264 - sparse_categorical_accuracy: 0.9187
Epoch 3/6
469/469 [==============================] - 2s 3ms/step - loss: 0.3482 - sparse_categorical_accuracy: 0.9351
Epoch 4/6
469/469 [==============================] - 2s 3ms/step - loss: 0.2304 - sparse_categorical_accuracy: 0.9487
Epoch 5/6
469/469 [==============================] - 2s 3ms/step - loss: 0.1748 - sparse_categorical_accuracy: 0.9576
Epoch 6/6
469/469 [==============================] - 2s 3ms/step - loss: 0.1485 - sparse_categorical_accuracy: 0.9641


In [8]:
def Ribli19(imsize=50, n_target=2 ,n_channels=1, nf=64, reg = 5e-5,
          padding='same', gpu='0'):

    #input
    inp = kl.Input((imsize, imsize,n_channels))
    
    # conv block 1
    x = kl.Conv2D(nf, (3, 3), padding=padding,kernel_regularizer=kr.l2(reg))(inp)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    
    x = kl.Conv2D(nf, (3, 3), padding=padding,kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    
    x = kl.MaxPooling2D(strides=(2,2))(x)
    
    # conv block 2
    x = kl.Conv2D(2*nf, (3, 3), padding=padding,kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    
    x = kl.Conv2D(2*nf, (3, 3), padding=padding, kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    
    x = kl.MaxPooling2D(strides=(2,2))(x)

    # conv block 3
    x = kl.Conv2D(4*nf, (3, 3), padding=padding, kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    
    x = kl.Conv2D(2*nf, (1, 1), padding=padding, kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    
    x = kl.Conv2D(4*nf, (3, 3), padding=padding, kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    x = kl.MaxPooling2D(strides=(2,2))(x)

    # conv block 4
    x = kl.Conv2D(8*nf, (3, 3), padding=padding, kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    
    x = kl.Conv2D(4*nf, (1, 1), padding=padding, kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    
    x = kl.Conv2D(8*nf, (3, 3), padding=padding, kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    x = kl.MaxPooling2D(strides=(2,2))(x)
    
    # conv block 5
    x = kl.Conv2D(16*nf, (3, 3), padding=padding, kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    
    x = kl.Conv2D(8*nf, (1, 1), padding=padding,  kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    
    x = kl.Conv2D(16*nf, (3, 3), padding=padding, kernel_regularizer=kr.l2(reg))(x)
    x = kl.Activation('relu')(kl.BatchNormalization()(x))
    
    #  end of conv
    
    x = kl.GlobalAveragePooling2D()(x)    
    x = kl.Dense(n_target, name = 'final_dense_n%d_ngpu%d' % (n_target, len(gpu.split(','))))(x)  

    model = km.Model(inputs=inp, outputs=x)  # make model
       
    return model

In [9]:
data = tfds.load('inverse_cat',as_supervised=True)
data = data['train']
data = data.cache()
data = data.shuffle(ds_info.splits['train'].num_examples)
data = data.batch(32)
data = data.prefetch(tf.data.experimental.AUTOTUNE)

In [10]:
model = Ribli19(50,n_channels=1)

In [12]:
sgd = ko.SGD(lr=0.005, decay=0., momentum=0.9, nesterov=True)
model.compile(loss='mse',optimizer=sgd, metrics=['mae','mse'])

model.fit(data,
          epochs=5,
          batch_size=32,
          verbose=1)

Epoch 1/40
3125/3125 [==============================] - 1916s 608ms/step - loss: 0.2017 - mae: 0.0583 - mse: 0.0098
Epoch 2/40
3125/3125 [==============================] - 1863s 596ms/step - loss: 0.1865 - mae: 0.0176 - mse: 4.9860e-04
Epoch 3/40
3125/3125 [==============================] - 1947s 623ms/step - loss: 0.1807 - mae: 0.0153 - mse: 3.7593e-04
Epoch 4/40
3125/3125 [==============================] - 1940s 621ms/step - loss: 0.1751 - mae: 0.0142 - mse: 3.2326e-04
Epoch 5/40
3125/3125 [==============================] - 1825s 584ms/step - loss: 0.1697 - mae: 0.0136 - mse: 2.9725e-04
Epoch 6/40
3125/3125 [==============================] - 1828s 585ms/step - loss: 0.1644 - mae: 0.0131 - mse: 2.7397e-04
Epoch 7/40
 228/3125 [=>............................] - ETA: 28:53 - loss: 0.1606 - mae: 0.0133 - mse: 2.8581e-04

KeyboardInterrupt: 